# Faster_RCNN model for the tasks

## 1. Introduction

1.  This model was trained for distinguishing of turtles or penguins and had a vital good result in the end.


2. <b>Basic Operating Environment of my implements:</b>
Python: 3.8.10 (default, Jun  4 2021, 15:09:15) [GCC 7.5.0]
CUDA available: True
GPU 0,1: NVIDIA GeForce RTX 2080 Ti
CUDA_HOME: /usr/local/cuda
NVCC: Cuda compilation tools, release 11.3, V11.3.109
GCC: gcc (Ubuntu 9.3.0-17ubuntu1~20.04) 9.3.0
PyTorch: 1.10.0+cu113
PyTorch compiling details: PyTorch built with:
- GCC 7.3
- C++ Version: 201402
- Intel(R) Math Kernel Library Version 2020.0.0 Product Build 20191122 for Intel(R) 64 architecture applications
- Intel(R) MKL-DNN v2.2.3 (Git Hash 7336ca9f055cf1bfa13efb658fe15dc9b41f0740)
- OpenMP 201511 (a.k.a. OpenMP 4.5)
- LAPACK is enabled (usually provided by MKL)
- NNPACK is enabled
- CPU capability usage: AVX512
- CUDA Runtime 11.3
- NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_50,code=sm_50;-gencode;arch=compute_60,code=sm_60;-gencode;arch=compute_70,code=sm_70;-gencode;arch=compute_75,code=sm_75;-gencode;arch=compute_80,code=sm_80;-gencode;arch=compute_86,code=sm_86
- CuDNN 8.2
- Magma 2.5.2
TorchVision: 0.11.1+cu113
OpenCV: 4.6.0
MMCV: 1.6.1
MMCV Compiler: GCC 9.3
MMCV CUDA Compiler: 11.3
MMDetection: 2.25.1+

## 2. Install Environment

In [ ]:
! pip install -U openmim

# Install MMEngine, an AI model training and inference engine
! mim install mmengine

# Install MMCV, a foundational library for computer vision research and it supports many research projects in the OpenMMLab ecosystem.
! mim install "mmcv>=2.0.0"

# Install MMDetection, an open-source toolbox for object detection in images
! mim install mmdet

In [ ]:
import mmdet
import mmcv

# Search for Faster R-CNN models in MMDetection's model zoo using OpenMIM
! mim search mmdet --model "faster r-cnn"

--------------------------------------------------------------------------------
config id: faster_rcnn_r50_caffe_c4_1x_coco
architecture                                            fpn,rpn,resnet,roipool
coco/box_ap                                             35.6
config                                                  configs/faster_rcnn/f...
epochs                                                  12
model                                                   faster r-cnn
paper                                                   URL,Title
readme                                                  configs/faster_rcnn/R...
training_data                                           coco
training_resources                                      8x v100 gpus
training_techniques                                     sgd with momentum,wei...
weight                                                  https://download.open...
--------------------------------------------------------------------------------
config

In [ ]:
# Use OpenMIM to download a pre-trained model

! mim download mmdet --config faster_rcnn_s50_fpn_syncbn-backbone+head_mstrain-range_1x_coco --dest .

## 3. DataSet Processing

   Extract data and transfer into the COCO format.




### 3.1 Picture extract

In [ ]:
import zipfile

 # Define the name of the file to be unzipped and the destination directory
zip_file_path = '/root/autodl-fs/Penguins_vs_Turtles.zip'
target_directory = 'dataset'

# Open the zip file and extract its content to the target directory
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(target_directory)

### 3.2 Annotations extract


In [ ]:
import json

# Load the annotation file, assuming it is a JSON list
with open('dataset/archive/train_annotations', 'r') as f:
    annotations_list = json.load(f)

annotations_dict = {item['id']: item for item in annotations_list}



### 3.2 Store the picture information & annotations information into the new COCO format file.

In [ ]:
import json
import os
from PIL import Image

def convert_to_coco_format(annotation_file, image_dir, output_file):

    with open(annotation_file, 'r') as f:
        annotations = json.load(f)

    images = []
    for image_name in os.listdir(image_dir):
        image_id = int(image_name.split('_')[2].split('.')[0])
        image = Image.open(os.path.join(image_dir, image_name))
        width, height = image.size
        images.append({
            'file_name': image_name,
            'height': height,
            'width': width,
            'id': image_id
        })

    categories = [
        {'id': 1, 'name': 'Penguin'},
        {'id': 2, 'name': 'Turtle'},

    ]

    coco_format = {
        'images': images,
        'annotations': annotations,
        'categories': categories
    }

    with open(output_file, 'w') as f:
        json.dump(coco_format, f)

# Now you can call this function for each of your datasets
convert_to_coco_format('dataset/archive/train_annotations', 'dataset/archive/train/train', 'train_coco_format.json')
convert_to_coco_format('dataset/archive/valid_annotations', 'dataset/archive/valid/valid', 'val_coco_format.json')


In [ ]:

with open('train_coco_format.json') as f:
  data = json.load(f)

# Modify the structure of the annotation
annotations = []
for i in range(len(data['images'])):
  annotations.append([])

for ann in data['annotations'][0]:
  annotations[ann['image_id']].append(ann)

data['annotations'] = annotations

with open('new_train_coco_format.json', 'w') as f:
  json.dump(data, f)

In [ ]:

def correct_image_ids(filename):
    # Load File
    with open(filename, 'r') as f:
        data = json.load(f)

    # Get Image ID and transfer them into the Int Type
    for image in data['images']:
        try:
            image['id'] = int(image['id'])
        except ValueError:
            print(f"Invalid 'id' in image: {image}")



# Process the COCO format file
correct_image_ids('train_coco_format.json')
correct_image_ids('val_coco_format.json')


## 4. Train Model Faster_RCNN from pretrain model

In [ ]:
! mim train mmdet turtle.py

Training command is python /root/mmdetection/mmdet/.mim/tools/train.py turtle.py --gpus 1 --launcher none. 
/root/mmdetection/mmdet/.mim/tools/train.py:155: UserWarning: `--gpus` is deprecated because we only support single GPU mode in non-distributed training. Use `gpus=1` now.
  warnings.warn('`--gpus` is deprecated because we only support '
fatal: not a git repository (or any of the parent directories): .git
2023-07-22 21:37:28,112 - mmdet - INFO - Environment info:
------------------------------------------------------------
sys.platform: linux
Python: 3.8.10 (default, Jun  4 2021, 15:09:15) [GCC 7.5.0]
CUDA available: True
GPU 0,1: NVIDIA GeForce RTX 2080 Ti
CUDA_HOME: /usr/local/cuda
NVCC: Cuda compilation tools, release 11.3, V11.3.109
GCC: gcc (Ubuntu 9.3.0-17ubuntu1~20.04) 9.3.0
PyTorch: 1.10.0+cu113
PyTorch compiling details: PyTorch built with:
  - GCC 7.3
  - C++ Version: 201402
  - Intel(R) Math Kernel Library Version 2020.0.0 Product Build 20191122 for Intel(R) 64 archite

In [ ]:
!mim test mmdet turtle.py --checkpoint work_dirs/turtle/latest.pth --show-dir output_test

Testing command is python /root/mmdetection/mmdet/.mim/tools/test.py turtle.py work_dirs/turtle/latest.pth --launcher none --show-dir output_test. 
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
load checkpoint from local path: work_dirs/turtle/latest.pth
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 72/72, 5.4 task/s, elapsed: 13s, ETA:     0sTesting finished successfully.


## 5. Valiadtion

![](valid_image/image_id_005.jpg)
![](valid_image/image_id_017.jpg)
![](valid_image/image_id_067.jpg)
![](valid_image/image_id_029.jpg)

In [ ]:
!mim test mmdet turtle.py --checkpoint work_dirs/turtle/latest.pth --out ./result/result_12.pkl --eval bbox --show

Testing command is python /root/mmdetection/mmdet/.mim/tools/test.py turtle.py work_dirs/turtle/latest.pth --launcher none --out ./result/result_12.pkl --eval bbox --show. 
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
load checkpoint from local path: work_dirs/turtle/latest.pth
[                                                  ] 0/72, elapsed: 0s, ETA:Figure(640.01x640.01)
[                                  ] 1/72, 1.6 task/s, elapsed: 1s, ETA:    44sFigure(640.01x640.01)
[                                  ] 2/72, 2.5 task/s, elapsed: 1s, ETA:    28sFigure(640.01x640.01)
[>                                 ] 3/72, 3.1 task/s, elapsed: 1s, ETA:    23sFigure(640.01x640.01)
[>                                 ] 4/72, 3.4 task/s, elapsed: 1s, ETA:    20sFigure(640.01x640.01)
[>>                                ] 5/72, 3.7 task/s, elapsed: 1s, ETA:    18sFigure(640.01x640.01)
[>>                                ] 6/72, 4.0 task/s, elapsed: 2s, ETA:    17sF

### 5.1 Extra prediction result
#### Compare the category id & bbox area

In [ ]:
import pickle
import mmcv
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
import numpy as np


# Load the configuration file
cfg = mmcv.Config.fromfile('turtle.py')

# Build the dataset
dataset = build_dataset(cfg.data.test)

# Load the results
with open('result/result_12.pkl', 'rb') as f:
    results = pickle.load(f)

import json

# Load the annotations file
with open('val_coco_format.json', 'r') as f:
    annotations = json.load(f)

# Get the ground truth labels
gt_labels = [ann['category_id'] for ann in annotations['annotations']]


loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


In [ ]:
# Get the predicted labels
import json
from mmdet.apis import init_detector, inference_detector
import os
from pycocotools.coco import COCO
from tqdm import tqdm
import numpy as np

anno_path = 'val_coco_format.json'
coco = COCO(anno_path)
ids = list(coco.imgs.keys())

filename_id = {}

# Get Image ID
for idx in tqdm(range(len(ids))):
    img_id = ids[idx]
    img_ids = coco.getImgIds(imgIds=img_id)
    image = coco.loadImgs(img_ids)
    for i in image:
        filename_id[i['file_name']] = i['id']


config_file = 'turtle.py'

# Trained Model

checkpoint_file =  "work_dirs/turtle/latest.pth"



model = init_detector(config=config_file,
                      checkpoint=checkpoint_file,
                      device='cuda:0')

root = 'dataset/archive/valid/valid/'

results = []

for file in tqdm(os.listdir(root)):
    result = inference_detector(model=model, imgs=root + file)
    for cate, items in enumerate(result, 1):
        for item in items:
            item = item.tolist()
            x, y, w, h, s = item[0], item[1], item[2], item[3], item[4]
            d = {}
            d['image_id'] = filename_id[file]
            d['category_id'] = cate
            d['bbox'] = [x, y, w, h]
            d['score'] = s
            results.append(d)

with open('result.json', 'w') as f:
    json.dump(results, f, indent=4)

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


100%|██████████| 72/72 [00:00<00:00, 226889.47it/s]


load checkpoint from local path: work_dirs/turtle/latest.pth


  0%|          | 0/72 [00:00<?, ?it/s]/root/mmdetection/mmdet/datasets/utils.py:66: UserWarning: "ImageToTensor" pipeline is replaced by "DefaultFormatBundle" for batch inference. It is recommended to manually replace it in the test data pipeline in your config file.
  warnings.warn(
100%|██████████| 72/72 [00:06<00:00, 10.78it/s]


In [ ]:
from pycocotools.coco import COCO
import numpy as np

def bbox_iou(boxA, boxB):
    # determine the (x, y)-coordinates of the intersection rectangle
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    interArea = max(0, xB - xA) * max(0, yB - yA)
    boxAArea = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
    boxBArea = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])


    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the intersection area
    iou = interArea / float(boxAArea + boxBArea - interArea)

    # return the intersection over union value
    return iou

def center_distance(boxA, boxB):
    # Calculate the center points
    centerA = ((boxA[0] + boxA[2]) / 2, (boxA[1] + boxA[3]) / 2)
    centerB = ((boxB[0] + boxB[2]) / 2, (boxB[1] + boxB[3]) / 2)

    # Calculate the distance between the center points
    distance = np.sqrt((centerA[0] - centerB[0]) ** 2 + (centerA[1] - centerB[1]) ** 2)
    
    return distance

# Create COCO format data
coco = COCO('val_coco_format.json')

img_ids = coco.getImgIds()

ious = []
distances = []


for i, img_id in enumerate(img_ids):

    ann_ids = coco.getAnnIds(imgIds=img_id)
    anns = coco.loadAnns(ann_ids)
    
    for ann in anns:
        # Save the real frame data
        gt_bbox = ann['bbox']  # [x,y,width,height]

        # Transfer into [xmin, ymin, xmax, ymax] format
        gt_bbox = [gt_bbox[0], gt_bbox[1], gt_bbox[0] + gt_bbox[2], gt_bbox[1] + gt_bbox[3]]
        
        # Get the corresponding predicted bounding box from the prediction results
        # Ensure the order of best_boxes is consistent with img_ids
        pred_bbox = best_boxes[i]

       # Calculate IoU
        iou = bbox_iou(gt_bbox, pred_bbox)
        ious.append(iou)
        
       # Calculate distance between center points
        distance = center_distance(gt_bbox, pred_bbox)
        distances.append(distance)


# Calculate mean and standard deviation of IoU

iou_mean = np.mean(ious)
iou_std = np.std(ious)

# Calculate mean and standard deviation of center distances
distance_mean = np.mean(distances)
distance_std = np.std(distances)

print('Mean IoU:', iou_mean)
print('Standard deviation of IoU:', iou_std)
print('Mean center distance:', distance_mean)
print('Standard deviation of center distance:', distance_std)


loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Mean IoU: 0.33019680706693144
Standard deviation of IoU: 0.18246687426322794
Mean center distance: 103.64549416035123
Standard deviation of center distance: 57.23252806262345


In [ ]:
!pip install scikit-learn
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
confusion = confusion_matrix(gt_labels, pred_labels)


# your ground truth labels and predicted labels
# gt_labels = ...
# pred_labels = ...

# Generate the classification report
report = classification_report(gt_labels, pred_labels)

report_dict = classification_report(gt_labels, pred_labels, output_dict=True)
precision = report_dict['weighted avg']['precision']
recall = report_dict['weighted avg']['recall']
f1_score = report_dict['weighted avg']['f1-score']
print(classification_report(gt_labels, pred_labels, labels=[1, 2]))


              precision    recall  f1-score   support

           1       0.97      0.94      0.96        36
           2       0.95      0.97      0.96        36

    accuracy                           0.96        72
   macro avg       0.96      0.96      0.96        72
weighted avg       0.96      0.96      0.96        72

